In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("Data - Original Data.csv", header=1)

# Linearly interpolate population in Scotland
df['SCT_Working_Pop'] = df['SCT_Working_Pop'].interpolate()
df['rUK_Working_Pop'] = df['rUK_Working_Pop'].interpolate()
df['Year'] = df['Year'].fillna(method='ffill')

# Limit sample to 1998Q1 to 2007Q4
df = df.query("1998 <= Year <= 2007")

df.head(5)

,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,UK_Compensation_of_Employees,rUK_Working_Pop,LIBOR
0,1998.0,1.0,65.1,19395.0,12520.0,9444.0,3281902.0,61.2685,245778.0,157147.0,111444,33910488.0,7.47
1,1998.0,2.0,65.4,19548.0,12635.0,9692.0,3281797.0,61.5281,248133.0,158994.0,113808,33961408.5,7.49
2,1998.0,3.0,65.3,19468.0,12828.0,9869.0,3281692.0,62.1079,251225.0,162342.0,115934,34012329.0,7.58
3,1998.0,4.0,65.1,19435.0,12892.0,9921.0,3281587.0,62.4503,254190.0,162810.0,116605,34063249.5,6.80
4,1999.0,1.0,65.5,19613.0,13074.0,10059.0,3281482.0,62.5442,256399.0,165491.0,119895,34114170.0,5.51


In [4]:
for var in ['Consumption', 'Output', 'Compensation_of_Employees']:
    df[f'rUK_{var}'] = df[f'UK_{var}'] - df[f'SCT_{var}']

df.head(5)

,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,UK_Compensation_of_Employees,rUK_Working_Pop,LIBOR,rUK_Consumption,rUK_Output,rUK_Compensation_of_Employees
0,1998.0,1.0,65.1,19395.0,12520.0,9444.0,3281902.0,61.2685,245778.0,157147.0,111444,33910488.0,7.47,144627.0,226383.0,102000.0
1,1998.0,2.0,65.4,19548.0,12635.0,9692.0,3281797.0,61.5281,248133.0,158994.0,113808,33961408.5,7.49,146359.0,228585.0,104116.0
2,1998.0,3.0,65.3,19468.0,12828.0,9869.0,3281692.0,62.1079,251225.0,162342.0,115934,34012329.0,7.58,149514.0,231757.0,106065.0
3,1998.0,4.0,65.1,19435.0,12892.0,9921.0,3281587.0,62.4503,254190.0,162810.0,116605,34063249.5,6.80,149918.0,234755.0,106684.0
4,1999.0,1.0,65.5,19613.0,13074.0,10059.0,3281482.0,62.5442,256399.0,165491.0,119895,34114170.0,5.51,152417.0,236786.0,109836.0


In [5]:
def adjust_series(series, defl, pop):
    return series / (defl * 100 * pop)

def diff_log(series):
    return np.log(series).diff()

sct_defl = df["SCT_Deflator (2018=100)"]
sct_pop = df["SCT_Working_Pop"]

adjusted_y = adjust_series(df["SCT_Output"], sct_defl, sct_pop)
diff_log_adjusted_y = diff_log(adjusted_y)
df["y_obs"] = diff_log_adjusted_y[1:] - np.average(diff_log_adjusted_y[1:])

adjusted_c = adjust_series(df["SCT_Consumption"], sct_defl, sct_pop)
diff_log_adjusted_c = diff_log(adjusted_c)
df["c_obs"] = diff_log_adjusted_c[1:] - np.average(diff_log_adjusted_c[1:])

adjusted_w = adjust_series(df["SCT_Compensation_of_Employees"], sct_defl, sct_pop)
diff_log_adjusted_w = diff_log(adjusted_w)
df["w_obs"] = diff_log_adjusted_w[1:] - np.average(diff_log_adjusted_w[1:])

log_pi = np.log(sct_defl)
diff_log_pi = log_pi.diff()
df["pi_obs"] = diff_log_pi[1:] - np.average(diff_log_pi[1:])

diff_interest = df["LIBOR"].diff()
df["i_obs"] = diff_interest[1:]

df

,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,...,rUK_Working_Pop,LIBOR,rUK_Consumption,rUK_Output,rUK_Compensation_of_Employees,y_obs,c_obs,w_obs,pi_obs,i_obs
0,1998.0,1.0,65.1,19395.0,12520.0,9444.0,3281902.00,61.2685,245778.0,157147.0,...,33910488.00,7.47,144627.0,226383.0,102000.0,NaN,NaN,NaN,NaN,NaN
1,1998.0,2.0,65.4,19548.0,12635.0,9692.0,3281797.00,61.5281,248133.0,158994.0,...,33961408.50,7.49,146359.0,228585.0,104116.0,-0.000568,0.000645,0.014851,-0.002452,0.02
2,1998.0,3.0,65.3,19468.0,12828.0,9869.0,3281692.00,62.1079,251225.0,162342.0,...,34012329.00,7.58,149514.0,231757.0,106065.0,-0.006398,0.012789,0.013156,-0.008580,0.09
3,1998.0,4.0,65.1,19435.0,12892.0,9921.0,3281587.00,62.4503,254190.0,162810.0,...,34063249.50,6.80,149918.0,234755.0,106684.0,-0.002457,0.004143,0.001851,-0.010117,-0.78
4,1999.0,1.0,65.5,19613.0,13074.0,10059.0,3281482.00,62.5442,256399.0,165491.0,...,34114170.00,5.51,152417.0,236786.0,109836.0,-0.000836,0.003992,0.001216,-0.000924,-1.29
5,1999.0,2.0,65.5,19584.0,13298.0,10350.0,3280784.50,62.7262,258184.0,167521.0,...,34170658.50,5.20,154223.0,238600.0,112520.0,-0.005127,0.013268,0.022227,-0.007049,-0.31
6,1999.0,3.0,65.9,19812.0,13283.0,10502.0,3280087.00,62.5541,261803.0,168041.0,...,34227147.00,5.19,154758.0,241991.0,114593.0,0.001840,-0.010937,0.002200,-0.000961,-0.01
7,1999.0,4.0,66.3,20148.0,13458.0,10610.0,3279389.50,63.0642,267764.0,169533.0,...,34283635.50,5.90,156075.0,247616.0,116262.0,0.007119,0.003317,-0.002112,-0.000998,0.71
8,2000.0,1.0,66.4,20383.0,13728.0,10623.0,3278692.00,63.4157,272383.0,172926.0,...,34340124.00,6.12,159198.0,252000.0,117535.0,0.006442,0.014637,-0.006574,-0.005542,0.22
9,2000.0,2.0,66.6,20416.0,13859.0,10886.0,3280680.25,63.4138,274159.0,174426.0,...,34402693.00,6.19,160567.0,253743.0,120354.0,-0.005856,0.001951,0.014338,-0.004042,0.07


In [6]:
def adjust_series(series, defl, pop):
    return series / (defl * 100 * pop)

def diff_log(series):
    return np.log(series).diff()

rUK_defl = df["UK_Deflator (2023=100)"]
rUK_pop = df["rUK_Working_Pop"]

adjusted_y = adjust_series(df["rUK_Output"], rUK_defl, rUK_pop)
diff_log_adjusted_y = diff_log(adjusted_y)
df["y_obs_ruk"] = diff_log_adjusted_y[1:] - np.average(diff_log_adjusted_y[1:])

adjusted_c = adjust_series(df["rUK_Consumption"], rUK_defl, rUK_pop)
diff_log_adjusted_c = diff_log(adjusted_c)
df["c_obs_ruk"] = diff_log_adjusted_c[1:] - np.average(diff_log_adjusted_c[1:])

adjusted_w = adjust_series(df["rUK_Compensation_of_Employees"], rUK_defl, rUK_pop)
diff_log_adjusted_w = diff_log(adjusted_w)
df["w_obs_ruk"] = diff_log_adjusted_w[1:] - np.average(diff_log_adjusted_w[1:])

log_pi = np.log(rUK_defl)
diff_log_pi = log_pi.diff()
df["pi_obs_ruk"] = diff_log_pi[1:] - np.average(diff_log_pi[1:])

df

,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,...,rUK_Compensation_of_Employees,y_obs,c_obs,w_obs,pi_obs,i_obs,y_obs_ruk,c_obs_ruk,w_obs_ruk,pi_obs_ruk
0,1998.0,1.0,65.1,19395.0,12520.0,9444.0,3281902.00,61.2685,245778.0,157147.0,...,102000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998.0,2.0,65.4,19548.0,12635.0,9692.0,3281797.00,61.5281,248133.0,158994.0,...,104116.0,-0.000568,0.000645,0.014851,-0.002452,0.02,-0.000498,0.003074,0.007695,-0.001256
2,1998.0,3.0,65.3,19468.0,12828.0,9869.0,3281692.00,62.1079,251225.0,162342.0,...,106065.0,-0.006398,0.012789,0.013156,-0.008580,0.09,-0.001545,0.007349,0.000560,0.003895
3,1998.0,4.0,65.1,19435.0,12892.0,9921.0,3281587.00,62.4503,254190.0,162810.0,...,106684.0,-0.002457,0.004143,0.001851,-0.010117,-0.78,0.001410,-0.007397,-0.008284,0.000014
4,1999.0,1.0,65.5,19613.0,13074.0,10059.0,3281482.00,62.5442,256399.0,165491.0,...,109836.0,-0.000836,0.003992,0.001216,-0.000924,-1.29,0.001169,0.010434,0.019012,-0.003982
5,1999.0,2.0,65.5,19584.0,13298.0,10350.0,3280784.50,62.7262,258184.0,167521.0,...,112520.0,-0.005127,0.013268,0.022227,-0.007049,-0.31,-0.001378,0.004118,0.012473,-0.002578
6,1999.0,3.0,65.9,19812.0,13283.0,10502.0,3280087.00,62.5541,261803.0,168041.0,...,114593.0,0.001840,-0.010937,0.002200,-0.000961,-0.01,0.010759,0.001457,0.012242,-0.008232
7,1999.0,4.0,66.3,20148.0,13458.0,10610.0,3279389.50,63.0642,267764.0,169533.0,...,116262.0,0.007119,0.003317,-0.002112,-0.000998,0.71,0.008759,-0.004398,-0.002420,0.002637
8,2000.0,1.0,66.4,20383.0,13728.0,10623.0,3278692.00,63.4157,272383.0,172926.0,...,117535.0,0.006442,0.014637,-0.006574,-0.005542,0.22,0.005896,0.009506,-0.003424,0.000074
9,2000.0,2.0,66.6,20416.0,13859.0,10886.0,3280680.25,63.4138,274159.0,174426.0,...,120354.0,-0.005856,0.001951,0.014338,-0.004042,0.07,0.000653,0.003671,0.014802,-0.005514


In [7]:
# Export for dynare

dynare_sct_data_cols = ["y_obs", "c_obs", "w_obs", "pi_obs"]
dynare_ruk_data_cols = ["y_obs_ruk", "c_obs_ruk", "w_obs_ruk", "pi_obs_ruk", "i_obs"]
dynare_data = df[dynare_sct_data_cols + dynare_ruk_data_cols][1:]
dynare_data.to_csv('../DynareData.csv', index=False)

dynare_data

,y_obs,c_obs,w_obs,pi_obs,y_obs_ruk,c_obs_ruk,w_obs_ruk,pi_obs_ruk,i_obs
1,-0.000568,0.000645,0.014851,-0.002452,-0.000498,0.003074,0.007695,-0.001256,0.02
2,-0.006398,0.012789,0.013156,-0.008580,-0.001545,0.007349,0.000560,0.003895,0.09
3,-0.002457,0.004143,0.001851,-0.010117,0.001410,-0.007397,-0.008284,0.000014,-0.78
4,-0.000836,0.003992,0.001216,-0.000924,0.001169,0.010434,0.019012,-0.003982,-1.29
5,-0.005127,0.013268,0.022227,-0.007049,-0.001378,0.004118,0.012473,-0.002578,-0.31
6,0.001840,-0.010937,0.002200,-0.000961,0.010759,0.001457,0.012242,-0.008232,-0.01
7,0.007119,0.003317,-0.002112,-0.000998,0.008759,-0.004398,-0.002420,0.002637,0.71
8,0.006442,0.014637,-0.006574,-0.005542,0.005896,0.009506,-0.003424,0.000074,0.22
9,-0.005856,0.001951,0.014338,-0.004042,0.000653,0.003671,0.014802,-0.005514,0.07
10,-0.003701,-0.004685,0.004428,0.000430,-0.001084,0.002997,0.008477,-0.001132,-0.07


In [16]:
import scipy.io as sio

mat = sio.loadmat("/Users/edvardasvabuolas/Documents/University/Dissertation/dissertation/metropolis/dissertation_mh1_blck1.mat")

mat_df = pd.DataFrame(mat['x2'])
mat_df = mat_df.iloc[:, 9:13]
mat_df.columns = ['chi_pi', 'chi_y', 'chi_pi_ruk', 'chi_y_ruk']
mat_df.to_csv("/Users/edvardasvabuolas/Documents/University/Dissertation/EstimatedParams.csv")
mat_df

,chi_pi,chi_y,chi_pi_ruk,chi_y_ruk
0,0.072849,0.052237,0.918778,0.871295
1,0.070871,0.059021,0.910351,0.871476
2,0.074043,0.054279,0.908058,0.861514
3,0.072489,0.046135,0.908260,0.863647
4,0.072489,0.046135,0.908260,0.863647
...,...,...,...,...
49995,0.097442,0.082174,0.915200,0.843642
49996,0.097442,0.082174,0.915200,0.843642
49997,0.097442,0.082174,0.915200,0.843642
49998,0.104803,0.080790,0.925210,0.855733


In [8]:
sct_pop_sum = sum(df["SCT_Working_Pop"])
uk_pop_sum = sum(df["rUK_Working_Pop"] + df["SCT_Working_Pop"])

sct_pop_share = sct_pop_sum / uk_pop_sum

sct_pop_share

0.08629149300230622